<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/02_Key_Value_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SparkContext 생성

In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)

# CSV 파일을 RDD로딩

In [ ]:
directory = "C:\\Users\\mhso_lec\\study_notebook\\data"
filename  = "restaurant_reviews.csv"

In [ ]:
lines = sc.textFile(f"file:///{directory}\\{filename}")
lines

file:///C:\Users\mhso_lec\study_notebook\data\restaurant_reviews.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

**header(첫 줄)빼고 전부 다 가져오기**

In [ ]:
header = lines.first()
header

'id,item,cateogry,reviews,'

In [ ]:
filtered_lines = lines.filter(lambda row : row != header)
filtered_lines

PythonRDD[3] at RDD at PythonRDD.scala:53

In [ ]:
filtered_lines.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [ ]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2]
    
    # reviews는 정수로 parse
    reviews = fields[3]
    reviews = int(reviews)
    
    return category, reviews

In [ ]:
parse('0,짜장면,중식,125,')

('중식', 125)

RDD 내의 모든 row에 대해 `parse` 함수를 적용 후 추출(`map`)

In [ ]:
category_reviews = filtered_lines.map(parse)
category_reviews

PythonRDD[4] at RDD at PythonRDD.scala:53

In [ ]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

카테고리 별 리뷰 평균
- 각 카테고리 별 리뷰의 개수
- 카테고리 종류의 개수
    - 중식은 몇 건, 분식은 몇 건...

In [ ]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
category_review_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

예를 들어 **중식**의 `x`, `y`

- 첫 번째 중식 : `(125, 1)` -> `x`
- 두 번째 중식 : `(235, 1)` -> `y`

------
- 리뷰의 개수끼리 합치기 : `x[0] + y[0]`
- 건수 끼리 합치기 : `x[1] + y[1]`



In [ ]:
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [ ]:
average = reduced.mapValues(lambda x : x[0] / x[1])
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [ ]:
sc.stop()